<a href="https://colab.research.google.com/github/ale-chen/Arbitrage/blob/main/FormatData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Goal:** *Predict Arbitrage Profit Margin for a Given Premier League Game*

---
**X:** (What we are training on)
- Data from Previous *5* games involving either team (total 10 games)
- Current Game Date, Time, Team IDs

**y:** (What we are predicting)
- Possible Profit Margin: Categorized into 1/4 classes, based on percent margin

In [1]:
import warnings
import os
import glob
warnings.filterwarnings("ignore")

In [2]:
os.chdir("drive/MyDrive/prem_data/data")

In [3]:
files = glob.glob("*.csv")

In [4]:
import pandas as pd

dfs = []

for file in files:
  try:
    dfs.append(pd.read_csv(file, parse_dates = ["Date"], dayfirst = True))
  except:
    print("Error intaking " + str(file))
    continue

In [5]:
#Check all columns align
if all([set(dfs[0].columns) == set(df.columns) for df in dfs]):
    print('All have the same columns')
else:
    print('Some have different columns')

Some have different columns


In [6]:
#Keep Only Match Stats, Market (Aggregate) Odds

odds_cols = ["B365%s","BS%s","BW%s","GB%s","IW%s","LB%s","PS%s",'P%s','SO%s','SB%s','SJ%s','SY%s','VC%s','WH%s']

home_odds_cols = [str(i) % "H" for i in odds_cols]
draw_odds_cols = [str(i) % "D" for i in odds_cols]
away_odds_cols = [str(i) % "A" for i in odds_cols]

In [7]:
dfsSlim = []

for df in dfs:
  dfsSlim.append(df.loc[:, [
      'Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','HTHG','HTAG',
      'HTR','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR'
      ]])

In [8]:
#For each row, each df get Max Odds for all 3 outcomes

for i, df in enumerate(dfs):
  maxH = []
  maxD = []
  maxA = []
  for index, row in df.iterrows():
    oddsH = []
    oddsD = []
    oddsA = []
    for colOdd in home_odds_cols:
      try:
        oddsH.append(row[colOdd])
      except:
        continue
    for colOdd in draw_odds_cols:
      try:
        oddsD.append(row[colOdd])
      except:
        continue
    for colOdd in away_odds_cols:
      try:
        oddsA.append(row[colOdd])
      except:
        continue
    maxH.append(max(oddsH))
    maxD.append(max(oddsD))
    maxA.append(max(oddsA))
  
  dfsSlim[i]['MAXH'] = maxH
  dfsSlim[i]['MAXD'] = maxD
  dfsSlim[i]['MAXA'] = maxA

In [9]:
#Combine all dfs
premier = pd.concat(dfsSlim)
premier['Date'] = pd.to_datetime(premier['Date']) #Reshuffle by date
premier = premier.sort_values(by='Date', ignore_index = True)

#Drop the single(?) na row
premier.loc[pd.isna(premier["MAXH"]), :].index
premier = premier[:-1]

teams = {}

for team in premier["HomeTeam"]: #Create dictionary of teams 
  if team not in teams.values():
    teams[len(teams)] = team
for team in premier["AwayTeam"]:
  if team not in teams.values():
    teams[len(teams)] = team

#teams but keys and vals flipped
teamsInv = dict([(value, key) for key, value in teams.items()])

premier["HomeTeam"] = premier["HomeTeam"].apply(lambda x: teamsInv[x])
premier["AwayTeam"] = premier["AwayTeam"].apply(lambda x: teamsInv[x])

outcome = {
    "H": -1,
    "D": 0,
    "A": 1
}
#for FTR and HTR, replace H/D/A with -1,0,1
premier["FTR"] = premier["FTR"].apply(lambda x: outcome[x])
premier["HTR"] = premier["HTR"].apply(lambda x: outcome[x])

#Reset Date to Ordinal to use as input data
premier["Date"] = premier["Date"].apply(lambda x: pd.Timestamp.toordinal(x))

In [10]:
from tabulate import tabulate

#GENERATE ONE ROW OF INPUT DATA
def gen_X(df: pd.DataFrame, index: int): #WILL THROW ERROR IF THERE ARE TOO FEW GAMES BEFORE INDEX
  hometeam = df['HomeTeam'][index]
  awayteam = df['AwayTeam'][index]

  homegames = []
  awaygames = []

  search = df.truncate(after=index)
#REVERSE ORDER IN ORDER TO SEARCH FROM CLOSEST TO FARTHEST
  search = search.loc[::-1]

#WILL THROW ERROR IF THERE ARE TOO FEW GAMES BEFORE INDEX
#REMEMBER TO CATCH LATER

  for i in range(len(search)):
    if(len(homegames) < 5):
      if df.iloc[i]['HomeTeam'] == hometeam or df.iloc[i]['AwayTeam'] == hometeam:
        homegames.append(df.iloc[[i]])
    if(len(awaygames) < 5):
      if df.iloc[i]['HomeTeam'] == awayteam or df.iloc[i]['AwayTeam'] == awayteam:
        awaygames.append(df.iloc[[i]])
    if(len(homegames) == 5 and len(awaygames) == 5):
      break
  """
  for i in search.iterrows():
    print(i)
    if(len(homegames) == 5 and len(homegames) == 5):
      break
    else:
      if i[1][1] == hometeam or i[1][2] == hometeam: #i[1][1] is home team
        homegames.append(i)
      if i[1][1] == awayteam or i[1][2] == awayteam: #i[1][2] is away team
        awaygames.append(i)
  """
#PREDICTOR CANNOT HAVE ACCESS TO CURRENT BETTING DATA FOR PREDICTION
  result = df.iloc[[index]].drop(columns = ['MAXH','MAXD','MAXA'])
  
  for index, game in enumerate(homegames):
    game.rename(columns = lambda s: s + "_H_" + str(index), inplace = True)
    result = pd.concat(
        [result.reset_index(drop=True),
         game.reset_index(drop=True)],
         axis = 1)
  
  for game in awaygames:
    game.rename(columns = lambda s: s + "_A_" + str(index), inplace = True)
    result = pd.concat(
        [result.reset_index(drop=True),
         game.reset_index(drop=True)],
         axis = 1)
  
  #print(tabulate(result, headers = 'keys', tablefmt = 'psql'))
  
  #Returns result, number of historical games as tuple
  return result, len(homegames) + len(awaygames)

In [11]:
#GENERATE ONE ROW OF TARGET DATA (Basically output arb level)
#We want to turn a regression problem into a classification problem
"""
idx | profit range
-------------------
-2:   [-inf, -2%]
-1:   [-2%, -0%]
1 :   [+0%, +2%]
2 :   [+2%, +inf]
"""
def gen_y(df: pd.DataFrame, index: int):
  maxH = df.iloc[index]["MAXH"]
  maxD = df.iloc[index]["MAXD"]
  maxA = df.iloc[index]["MAXA"]
  
  profit = arbProfit(maxH, maxD, maxA)
# UNCOMMENT THIS CODE BLOCK TO RETURN CATEGORICAL DATA
  """
  if profit < -.02:
    result = -2
  elif -.02 <= profit < 0:
    result = -1
  elif 0 <= profit < .02:
    result = 1
  else:
    result = 2
  """
# COMMENT OUT THIS CODE BLOCK TO RETURN CATEGORICAL DATA
  result = profit

  return result

#Intakes Odds, returns unbiased profit margin
def arbProfit(H,D,A):
  impliedWin = (1/H) + (1/D) + (1/A)
  return (1/impliedWin) - 1 

#For fun, returns bet amounts to result in arbProfit outcome
def betAmt(investment, H,D,A):
  impliedWin = (1/H) + (1/D) + (1/A)
  hAmt = (investment * (1/H)) / impliedWin
  dAmt = (investment * (1/D)) / impliedWin
  aAmt = (investment * (1/A)) / impliedWin

  return hAmt, dAmt, aAmt

In [12]:
import numpy as np

X = []
y = []

"""
PROCESS ALL DATA, ONLY APPEND DATA WHICH HAS 10 PREVIOUS GAMES
EXPORT TO CSV
THIS TOOK *30 MINUTES* TO COMPLETE
"""

for i in range(len(premier)):
  if i % 100 == 0:
    print("Processed next 100 rows; on row #" + str(i))
  Xline, trainGames = gen_X(premier, i)
  if(trainGames == 10):
    X.append(Xline.loc[0, :].values.flatten().tolist())
    y.append(gen_y(premier, i))

import csv

with open("X.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(X)
with open("y.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerow(y)

"""
y = []
for i in range(len(premier)):
  y.append(gen_y(premier, i))
unique, counts = np.unique(y, return_counts=True)
print(dict(zip(unique, counts)))

print(tabulate(premier.iloc[[1210]], headers = 'keys', tablefmt = 'psql'))
len(premier)
"""

Processed next 100 rows; on row #0
Processed next 100 rows; on row #100
Processed next 100 rows; on row #200
Processed next 100 rows; on row #300
Processed next 100 rows; on row #400
Processed next 100 rows; on row #500
Processed next 100 rows; on row #600
Processed next 100 rows; on row #700
Processed next 100 rows; on row #800
Processed next 100 rows; on row #900
Processed next 100 rows; on row #1000
Processed next 100 rows; on row #1100
Processed next 100 rows; on row #1200
Processed next 100 rows; on row #1300
Processed next 100 rows; on row #1400
Processed next 100 rows; on row #1500
Processed next 100 rows; on row #1600
Processed next 100 rows; on row #1700
Processed next 100 rows; on row #1800
Processed next 100 rows; on row #1900
Processed next 100 rows; on row #2000
Processed next 100 rows; on row #2100
Processed next 100 rows; on row #2200
Processed next 100 rows; on row #2300
Processed next 100 rows; on row #2400
Processed next 100 rows; on row #2500
Processed next 100 rows;

"\ny = []\nfor i in range(len(premier)):\n  y.append(gen_y(premier, i))\nunique, counts = np.unique(y, return_counts=True)\nprint(dict(zip(unique, counts)))\n\nprint(tabulate(premier.iloc[[1210]], headers = 'keys', tablefmt = 'psql'))\nlen(premier)\n"

In [13]:
premier.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY',
       'HR', 'AR', 'MAXH', 'MAXD', 'MAXA'],
      dtype='object')